# Introduction to RNA Sequencing
Welcome to the RNA-sequencing workshop for SSRP. This notebook will cover step-by-step how to perform a basic RNA-seq experiment. 

As this is built to operate on our [Binderhub](https://binderhub.readthedocs.io/en/latest/index.html), the environment has been preconfigured with appropriate software installed. These softwares will be called as needed. To view them, please see the environment file within the binder directory. If you try to run this notebook on a local machine without properly installing each software package, it will not operate correctly. 

Since this Jupyter kernel is based on Python, the base code for each block is Python unless defined otherwise. Text will be generated in markdown blocks, but otherwise look for %% to then define the language for the block. Such as a block starting with "%%bash", it will be executed as if it were the standard command line interface terminal, or "%%R", executes as if it were R code. 

## Primary analysis
Primary analysis entails the first line of analysis, in this case the very first step will have already been performed by your sequencing service provider- demultiplexing. This step converts the native Illumina base calls into the FASTQ format, while separating out different samples based upon their respective index. Per sample you should have:
- SampleName_R1.fastq.gz
- SampleName_R2.fastq.gz

The .gz at the end is a [gzip](https://www.gzip.org/) compression to reduce the overall file size. 

In this case, we will be utilizing publically available data. To that end, we would traditionally need to download the files through the [Short Reads Archive(SRA)](https://www.ncbi.nlm.nih.gov/sra). This would involve using their SRA toolkit, which has already been installed in this environment and used to download the files. The code would look like:
```bash
fastq-dump --split-files --gzip --outdir rawfiles/ SAMN09354753
```
Which would need to be done for each sample. The --split-files generates both R1 and R2, --gzip allows for continued compression, and --outdir notes the output directory for the files. 

Unfortunately, the SRA toolkit can be rather finnicky and doesnt like to operate consistently. In some cases it can be easier to pull the files from the [European Nucleotide Archive (ENA)](https://www.ebi.ac.uk/ena). For this, you need to find the FTP path, but then you can just use the standard unix tool wget to download the files. 
```bash
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR726/008/SRR7261718/SRR7261718_1.fastq.gz
```
For expediency- this has been included during the notebook initialization, and all samples should be renamed to fit the appropriate name below. Each sample should have the name followed by an underscore and then a 1 or 2. The 1 or 2 denotes if it was Read1 or Read2. 

For this workshop, we will be using data from [GSE115330](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE115330). This study was done to investigate the effects of various pollutions on the transcriptome of *Escherichia coli*. Each sample's RNA was isolated via trizol extraction, followed by ribosomal depeletion with the RiboZero kit, libraries generated by the TruSeq RNA library kit (sonic fragmentation), and lastly sequenced on an Illumina HiSeq 4000 in a 2x150bp configuration. 

**Sample breakdown table**

|  SRA Sample Name | SRA Run Name | Sample Name | Sample Type |
| --- | --- | --- | --- |
| SAMN09354753 | SRR7261718 | WT1 | Wild type |
| SAMN09354752 | SRR7261719 | WT2 | Wild type |
| SAMN09354751 | SRR7261720 | WT3 | Wild type |
| SAMN09354750 | SRR7261721 | Urban1 | Treated with collected urban air |
| SAMN09354749 | SRR7261722 | Urban2 | Treated with collected urban air |
| SAMN09354748 | SRR7261723 | Urban3 | Treated with collected urban air |
| SAMN09354747 | SRR7261724 | Diesel1 | Treated with collected diesel exhaust |
| SAMN09354746 | SRR7261725 | Diesel2 | Treated with collected diesel exhaust |
| SAMN09354745 | SRR7261726 | Diesel3 | Treated with collected diesel exhaust |

After receiving these files from your sequencing service provider (or in this case, downloading them), the next step in primary analysis is checking the quality of the reads. This follows the simple principle of **Garbage in=Garbage out** and always drives the need for the highest quality input available.

### FASTQ quality check
To check your FASTQ files, we will use FASTQC which is published by the [Babraham Bioinformatics](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/) team as the first line QC tool.

This tool will evaluate the overall health of your sample. In particular, you will want to look at:
- Total number of sequences
    - Make sure it aligns with your targetted expectations
- Per base sequencing quality
    - It should drop over time, especially in 2x300bp reads. Ensure that it doesnt drop too far, usually you want Q30>90% if possible
- QC content
    - Make sure it aligns with your genome of interest
- %N
    - N is an ambiguous base. You dont want many of these in your sample (if any)
- Sequence Length
    - Make sure the length you put in to the sequencer is whats coming out
- Duplication levels
    - Keep as low as possible, but any library with PCR you expect some
- Adapter content
    - This will be driven by your insert size and sequencing length. If you are sequencing through your entire insert and in to the adapter on the other side, you would expect contamination here. Usually best to avoid that, but if you see some you will need to trim it out. 

In [ ]:
%%bash
for file in raw_files/*.fastq.gz
do 
    fastqc $file &
done

Now if you didnt want to do that manually, you could set up a loop to iterate through and do this on all files that end in ".fastq.gz" as Ive done. As an example, here is a small code snippet to show how it would be done indivudally. Try it out if you would like!

```bash
fastqc raw_files/WT1_1.fastq.gz
```

To view your FASTQC output, navigate into the raw_align directory (containing the raw files for alignment) and look for .html files that are the output. Download those and then open them in your browser to explore the output.

### Read Trimming
After checking the input quality of the samples, you often (or always) will need to trim the samples. This can be critical because again, **Garbage in=Garbage out**. 
This step remove ambiguous bases, low quality bases/reads, and also removes any adapter content that may be in the read. For most trimming cases, [Trimmomatic](http://www.usadellab.org/cms/?page=trimmomatic) is a viable tool. Another commonly used tool is [CutAdapt](https://cutadapt.readthedocs.io/en/stable/), but here we will use trimmomatic. 

Parameters used:
- PE
    - Denotes the input as paired end reads. Following that, you need to put the two input files, then trimmed-1-unpaired, trimmed-1-paired, trimmed-2-unpaired, trimmed-2-paired for output files
- ILLUMINACLIP
    - To clip off specifically any Illumina reads that are found. The numbers that follow are the seed mismatch (maximum count of mismatches to identify adapter), palindrome clip threshold (to remove possible identical adapters on both ends of reads), and the simple clip threshold (how accurate the adapter is to read beyond the seed)
- LEADING TRAILING
    - Removes bases from the start and end of the read if they are below thresholds. "2" is Illumina for "low quality"
- SLIDING WINDOW
    - Looks at the read in a sliding window frame and takes the average of multiple bases. The first number is the window size and the second is quality score. Again "2" is Illumina for "low quality", but this allows for 1 base to not potentially trash an otherwise high quality read. 
- MINLEN
    - The minimum length of a read after trimming to accept.  

In this case, I opted to use a quick bash loop. The first line looks through the raw files directory, then pulls out the sample name from the fastq files. We then can use that to build the aspects of the trimmomatic command. 

In [ ]:
%%bash
mkdir trimmed
for prefix in $(ls raw_files/*.fastq.gz | sed -r 's/_[12][.]fastq.gz//' | uniq)
do
    output=$(echo $prefix | cut -d'/' -f2-)
    trimmomatic PE ${prefix}_1.fastq.gz ${prefix}_2.fastq.gz \
    trimmed/${output}_trimmed_1_paired.fastq.gz trimmed/${output}_trimmed_1_unpaired.fastq.gz \
    trimmed/${output}_trimmed_2_paired.fastq.gz trimmed/${output}_trimmed_2_unpaired.fastq.gz \
    ILLUMINACLIP:raw_files/adapters.fasta:2:40:15 \
    LEADING:2 TRAILING:2 \
    SLIDINGWINDOW:4:2 \
    MINLEN:140 &
done

And then do a QC check on the samples again to see how they are after trimming!

In [ ]:
%%bash
for file in trimmed/*.fastq.gz
do 
    fastqc $file &
done

And with that you have checked the quality of your samples, removed low quality contaminants, and can show the hopefully high-quality input for your secondary analysis!
We also introduce a tool called [MultiQC](https://multiqc.info/). This is a great QC aggregation tool. Play around with it a bit, but after running this you can download the multiqc file, open it in a browser, and view things aggregated instead of all the individual files. 

In [ ]:
%%bash
multiqc trimmed/.

## Secondary analysis
So secondary analysis is the data reduction step of an NGS workflow. Specifically to RNA-sequencing, this is where we would align the reads to the reference and assess the quality of alignment. There are many different aligners out there, but the one that has been the gold standard for RNA-sequencing has been [STAR](https://github.com/alexdobin/STAR). This aligner is accurate, fast, and splice-aware (for applicable organisms). 

### Build STAR reference index
The first step in performing STAR alignment is generating a reference index. The STAR algorithm uses a seed searching technique on an uncompressed suffix array, so requires indexing. 

During the notebook initilization, the reference genome and transcript tracks should have been downloaded as well into the references directory. For reference, we will be using the [K-12 strain of *E.coli*](http://bacteria.ensembl.org/Escherichia_coli_str_k_12_substr_mg1655/Info/Index).

In [ ]:
%%bash
mkdir star
STAR --runMode genomeGenerate \
    --sjdbGTFfile references/ecoli_genes.gtf \
    --sjdbGTFtagExonParentTranscript Parent \
    --genomeSAindexNbases 10 \
    --genomeDir star/ \
    --genomeFastaFiles references/ecoli.fasta

### Align reads to reference
Now that your reference is built appropriately for STAR, you can align your reads to it. This will be the longest steps of the processing. Below is a standard command for calling STAR. There are a variety of additional parameters you can pass to tweak the operations. These would generally be more advanced applications, and involve altering the seeding/extensions/scoring. For here though, we are going to make a directory for the output, pull our sample names again into a loop, and align each sample. The last step is to index the output file as well- this makes it easier to import into visualizing tools. 

In [ ]:
%%bash
mkdir aligned
for prefix in $(ls trimmed/*.fastq.gz | sed -r 's/_[12]_paired[.]fastq.gz//' | uniq)
do
    output=$(echo $prefix | cut -d'/' -f2-)
    STAR --genomeDir star/ \
            --sjdbGTFfile references/ecoli_genes.gtf \
            --readFilesIn ${prefix}_1_paired.fastq.gz ${prefix}_2_paired.fastq.gz \
            --twopassMode Basic \
            --outWigType bedGraph \
            --outSAMtype BAM SortedByCoordinate \
            --readFilesCommand zcat \
            --outFileNamePrefix aligned/${output}
    samtools index aligned/${output}Aligned.sortedByCoord.out.bam
done

### Post alignment QC
Now as has been mentioned several times, Garbage in=Garbage out. This is where we can more in depth evaluate the output quality to ensure that the different samples were successful throughout the entire workflow. We will use several tools for generating different aspects of QC, and then pull them together using the MultiQC tool. 

#### RSeqQC
The first tool we will use for the post run QC is called [RSeqQC](http://rseqc.sourceforge.net/). We will use several of its scripts to measure different aspects of our aligned data. You'll notice each of these scripts ends in ".py". This notes it as a python script- this program is built in python. 
- bam_stat.py: This script pulls the basic mapping statistics from the aligned file. This is the information like number of mapped reads, which were in proper pairs, which spanned splice junctions etc
- inner_distance.py: This script attempts to calculate the average inner distance of your fragments. This is the gap space between your reads. If your reads overlap, it should be zero. If you had say 400 bp fragments and did 2x100bp sequencing, you would expect an inner distance of ~200bp.  
- read_distribution.py: This script takes a look at where your reads were mapping. Was the read in an exonic region, intronic etc. 
- read_duplication.py: This script looks at the duplication of reads. In most cases of RNA-seq since there is amplification there is some expected duplication. If it is too high though, this means you probably overamplified and ended up sequencing the same tiny fragments. 

In [ ]:
%%bash
mkdir out_qc
for file in $(ls aligned/*_trimmedAligned.sortedByCoord.out.bam)
do
    name=$(echo $file | cut -d'/' -f2-)
    bam_stat.py -i $file 2> out_qc/${name}.bam_stat.txt
    inner_distance.py -i $file -o out_qc/${name}.rseqc -r $bed12
    read_distribution.py -i $file -r ${name} > out_qc/${name}.read_distribution.txt
    read_duplication.py -i $file -o out_qc/${name}.read_duplication
done

#### FeatureCounts
The next tool we will use is called [featureCounts](http://subread.sourceforge.net/), part of the Subread package. Similar to calculating the read distribution above, it also counts the reads that are aligned to each region. This gives us the raw hit counts data we need to perform the differential gene expression analysis. 

In [ ]:
%%bash
for file in $(ls aligned/*_trimmedAligned.sortedByCoord.out.bam)
do
    name=$(echo $file | cut -d'/' -f2-)
    featureCounts -a references/ecoli_genes.gtf -g 'gene_id' -t 'exon' -o counts/${name}_gene.featureCounts.txt --extraAttributes 'gene_name' -p $file
done

Now you'll have a bunch of individual featureCount files. At this point you'll need to merge them together to be able to perform downstream workflows. You can do this merging a variety of ways, using a python loop, some R. I opted in this case to use a quick bit of bash below. It navigates into the counts directory we made above, lists all of the files so it can then "clean" them up to be just the values for each gene, gets the gene names, then pastes it all together. The output is then a tab delimited spreadsheet. 

In [ ]:
%%bash
cd counts
ls -1 *featureCounts.txt | parallel 'cat {} | sed '1d' | cut -f8 {} > {/.}_clean.txt'
ls -1 *featureCounts.txt | head -1 | xargs cut -f1 > genes.txt
paste genes.txt *featureCounts_clean.txt > merged_gene_counts.txt
sed -i 1d merged_gene_counts.txt
cd ..
#Sometimes you may have the massage the data a bit at this point to get it to correctly enter the DESeq portions.
#I may have a typo or bug somewhere in here, so if it isnt perfect dont be alarmed- mistakes can happen. 

# Start Here for the quick version!

## Tertiary analysis
Now that we have determined we have a quality mapping and we have our table of gene counts, we are ready for the next step- differential gene expression analysis. One of the most prolificly utilized is [DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html). 

For this workshop, Ive included the feature counts file we would have generated above to save time, as it can take more time than we have to run the above steps. If there are any bugs in the above steps as well, then this will circumvent that as you now will have the appropriate files for downstream. 

Specifically for RNA-seq experiments like this, we need to take into account several aspects of the biology at play for the statistics. Primarily, a standard t-test assumes a normal distribution. Like a coin flip- If you flip a coin 50 times per test and do 50 test, you would expect the brunt of the output to be near the 25 heads/25 tails, with it being less and less likely to get 1 heads/49 tails or 1 tails/49 heads. That would be a normal distribution. With RNA-seq, we expect to see thousands of "tests" (genes) with many of them having few coin flips (reads) and then a select few having a larger portion of reads. This is more comparable to the lottery, many many will play, some will win a little bit, and very few will hit the jackpot. This type of distribution is usually called a Poisson distribution. RNA-seq goes even a bit further and no longer assumes that the mean and variance will be the same, and instead assumes that the variance is independent of the mean. This is called negative binomial distribution, and as you can see below fits an RNA-seq distribution. 

![https://biohpc.cornell.edu/doc/RNA-Seq-2019-Lecture3.pdf](img/nb_mean_var.png)

You also have to consider how to normalize your data. This is critical in RNA-seq data for the following effects:
- Significance due to different sequencing depth. If Sample A is twice as deep as Sample B, well then the genes will look twice as expressed
- Significance due to gene size. If Gene X is twice as long as Gene Y, you would expect more reads to map to Gene X since more of the original RNA would come from the larger gene, and therefore Gene X will look more expressed than Gene Y
- Significance due to RNA depth/composition. There will be several genes that are the most expressed based on the negative binomial distribution. These "super signals" will reduce the visibility of other signals. I always think of a car with those stupid bright HID LED lights driving at night looks like the light of the sun, and then the next car with normal headlights looks like its lights arent even on. Similar concept. 

With DESeq2, we fortunately can take into account these normalizations which we will go into a bit more below. 

Here, we will begin to use a different language than the bash above, and instead will use R for performing the stats and basic visualization. You can see each block will be run as R by the "%%R" that starts the block, like the "%%bash" noted bash above. 

The first step is loading the DESeq2 library and loading in the raw data. For this, we have pre-built the metadata table saying what each sample is and the comparators. During the first bit we will load all three samples, then we will walk back a few steps and repeat to perform the appropriate pairwise comparisons. 

In [ ]:
%%bash
echo ",SampleName,Type" >> meta.csv
echo "WT1,WT1,WildType" >> meta.csv
echo "WT2,WT2,WildType" >> meta.csv
echo "WT3,WT3,WildType" >> meta.csv
echo "Urban1,Urban1,Urban" >> meta.csv
echo "Urban2,Urban2,Urban" >> meta.csv
echo "Urban3,Urban3,Urban" >> meta.csv
echo "Diesel1,Diesel1,Diesel" >> meta.csv
echo "Diesel2,Diesel2,Diesel" >> meta.csv
echo "Diesel3,Diesel3,Diesel" >> meta.csv

In [ ]:
import rpy2.robjects as robjects
%load_ext rpy2.ipython

In [ ]:
%%R
#load the library for DESeq2
suppressMessages(library(DESeq2)) 

#Set up your parameters for this, in this case a path to your counts, your metadata, and which column in the metadata for cross comparisons. 
params <- list(feature_counts='counts/merged_gene_counts.txt', 
               annotation='meta.csv',
               condition='Type')

#Load your count file, it should be a tab delimited text file with headers. 
counts <- read.csv(params$feature_counts,
                       sep              = "\t",
                       header           = TRUE, 
                       stringsAsFactors = FALSE, 
                       check.names      = FALSE);

#Get the rownames, order them, and shift it over
rownames(counts) <- counts[,1]
counts <- counts[, -c(1:2)]
counts <- counts[ , order(names(counts))]
#Load in your metadata and likewise order it all to pair sample names in the metadata to the names in the counts file
sampleTable <- read.csv(params$annotation, row.names = 1)
sampleTable <- sampleTable[ order(row.names(sampleTable)), ]
countdata <- as.matrix(counts)[, colnames(counts) %in% rownames(sampleTable)]
condition <- unlist(sampleTable[params$condition])

#load a DESeq Data Set (dds for variable name) based on the inputs above. 
dds <- DESeqDataSetFromMatrix(countData=countdata, colData=sampleTable, design= as.formula(paste("~", params$condition)))

Now that you've loaded your data and got it inputted correctly, you need to consider how DESeq2 performs differential gene expression analysis and the normalization mentioned above. You will need to:
- Model the raw counts for each gene
    - Estimate size factors. Normalizes the data using a median of ratios method
    - Estimate gene-wise dispersion. The spread of variability in the data. 
    - Fit and shrink to gene-wise dispersion. For the RNA-depth/composition comparison, assumes that similar expression levels have similar dispersion and can be shrunk together to the generalized fit.
    - Generalized Linear Model fit for each gene
- Shrink log2 fold changes if applicable. This wont change the output stats, but will reduce the variance of the samples to the means of the gene, and can make for different visualizations.
- Statistically test for differential gene expression (more detail below)

As briefly touched on above, this will help you normalize the data to take into account sequencing depth, gene size, and RNA depth/composition. All together, this starts to look like:
![https://github.com/hbctraining/DGE_workshop_salmon/blob/master/img/deseq_dispersion2.png](img/deseq_dispersion2.png)

You can see how it brings the normalized counts onto a properly fit dispersion pattern. 

Now for the statistics, DESeq2 traditionally uses the Wald test. The Wald test specifically takes into account the dispersion model distance and the negative binomial distribution that we experience with RNA-seq. We then assume our null hypothesis is that there is no differential gene expression between the groups, and p is small, we can reject the null and say the gene is differentially expressed. One last thing to consider with the p-value is multiple testing correction. Traditionally p < 0.05 is considered significant (95% chance it is not by random chance), but when you are looking at thousands of genes, you'll still end up with 5% of them significant off predicted chance. One of the best corrections to use called False-Discovery Rate correction (FDR), which is the Benjamini and Hochberg method. This corrects the p-value based upon ranking the genes and accounting for the number of genes in the dataset. If you want to be even more conservative, you can use a bonferroni correction, but here we use FDR.

In the end one of the great things about DESeq2, is that it covers each of these aspects and if you manually want to adjust them you can, but it will automatically perform each of these steps with a simple command:

In [ ]:
%%R
dds <- DESeq(dds)

Yup, as simple as that to start off. Now lets take a look at the dispersion we see in our samples:

In [ ]:
%%R
plotDispEsts(dds, main="Dispersion plot")

Now at this step it also is good practice to look at how related samples are, which can also act as a mild QC to check that there werent any obvious sample mix ups. In this case, we will do a rlog transformation on the data for visualization purposes, and then we will do a sample relationship distance matrix and PCA plot. This code will be shown and lightly annotated. In R and Python, the "#" notes an annotation (or comment). Anything after "#" will not be run as code in that line. 

In [ ]:
%%R
rld <- rlogTransformation(dds)
suppressMessages(library(RColorBrewer)) #An R library to add some fancy colors
suppressMessages(library(gplots))
(mycols <- brewer.pal(8, "Dark2")[1:length(unique(condition))]) #Setting colors to the samples
sampleDists <- as.matrix(dist(t(assay(rld)))) #Calculating the distance between each sample as a matrix
heatmap.2(as.matrix(sampleDists), key=F, trace="none",
          col=colorpanel(100, "black", "white"),
          ColSideColors=mycols[condition], RowSideColors=mycols[condition],
         main="Day 14 Distance Matrix",
         margins=c(12,12),
         srtCol=45,
         srtRow=45)

In [ ]:
%%R
suppressMessages(library(ggplot2)); suppressMessages(library(ggrepel)); suppressMessages(library(dplyr))
rv = rowVars(assay(rld)) #Assign the row variables
select = order(rv, decreasing = TRUE)[seq_len(min(500, length(rv)))] #select the top facets for PCA
pca = prcomp(t(assay(rld)[select, ])) #execute PCA. This can be done with a few methods that are explored more in depth for a full course
data <- plotPCA(rld, intgroup=params$condition, returnData=TRUE) #generate the initial PCA plot
pc1var <- round(summary(pca)$importance[2,1]*100, digits=1) #Set the variable numbers for the label
pc2var <- round(summary(pca)$importance[2,2]*100, digits=1)
pc1lab <- paste0("PC1 (",as.character(pc1var),"%)") #and now make the label
pc2lab <- paste0("PC1 (",as.character(pc2var),"%)")
#all the ggplot details to add labels and make it look pretty. 
p1 <- ggplot(data, aes(PC1, PC2))
p1 <- p1 + geom_point(aes(color=mycols[condition], shape=mycols[condition]),alpha=0.55,size=5)
p1 <- p1 + scale_shape_manual(values=c(15, 17, 19, 18))
p1 <- p1 + xlab(pc1lab)
p1 <- p1 + ylab(pc2lab)
p1 <- p1 + geom_text_repel(aes(label=rownames(data)))
p1 <- p1 + theme(legend.position = "none")
print(p1)

We can now take a look at how related the samples are in the plots above. Each of the samples per group should be fairly close together, and you can see variance from group to group. 

The last thing we will do here is organize the output tables to show the appropriate cross comparisons. Going back to the biology, we will do the different conditions vs the wild type. Since this is for visualization purposes as well, we will implement the log fold change shrinkage mentioned above. 

In [ ]:
%%R
#Set up comparisons based on the metadata table
contrast_urban <- c('Type', 'Urban','WildType')
contrast_diesel <- c('Type','Diesel','WildType')

#Pull the results from the dataset and the appropriate contrasts
result_table_urban_preshrink <- results(dds, contrast=contrast_urban, alpha=0.05)
result_table_diesel_preshrink <- results(dds, contrast=contrast_diesel, alpha=0.05)

#Perform the lfc shrinkage. 
urban_out <- lfcShrink(dds, contrast=contrast_urban, res=result_table_urban_preshrink)
diesel_out <- lfcShrink(dds, contrast=contrast_diesel, res=result_table_diesel_preshrink)

#Make a quick directory to store the output files
dir.create('./DEG/')#DEG stands for differentially expressed genes

#Write it out as a csv file for your later pleasure
write.csv(as.data.frame(urban_out), file='./DEG/urban_output.csv')
write.csv(as.data.frame(diesel_out), file='./DEG/diesel_output.csv')
write.csv(as.data.frame(counts(dds, normalized=TRUE)), file='./DEG/full_counts.csv')

## Quartenary analysis
Now the last stages of analysis are also the most open ended- and this is considered quartenary analysis. This is where you now start to connect all the hard work you've done into some biological conclusions. In a lot of cases you'll already have some aspect you are focusing in on, maybe like the role of TH1 genes in your immune reponse to drug X or something, but other times you are purely on a fishing expedition. So here Ill leave it a bit more open ended- how would you start to explore this data? 

You could look at specific genes of interest, look at just the top genes, start plotting them out in different fashions. Ill give a few examples below, but then start to explore the data and come up with some biological conclusions. 

**Spoiler alert**, this paper was already published so if you really come up dry, go look there for inspiration. 

### Basic Plotting Options
Here Ill give some quick demonstrations of common plotting methods and things you might look at. 

#### Volcano plots
Volcano plots are a staple in RNA-seq analysis. These are generally used to look at the spread of differentially expressed genes between two samples. They are usually structured to show the high count of insignificant genes in on the bottom, then the more significant and alternatively regulated genes exploding out- looking a bit like a [plinian volcanic eruption](https://www.youtube.com/watch?v=VBTAcACmcgo) (hence the name- volcano plots). 

You can create an quick and dirty volcano plot using the code below, but if you want to get fancy with it I would recommend the [EnhancedVolcano](https://bioconductor.org/packages/release/bioc/html/EnhancedVolcano.html) package. If you want to get real hardcore, combine it with [plotly](https://plotly.com/) for interactive plots. 

In [ ]:
%%R
res <- read.csv("./DEG/urban_output.csv", header=TRUE, row.names=1)

# Make a basic volcano plot
with(res, plot(log2FoldChange, -log10(pvalue), pch=20, 
               main="Urban Atmosphere vs Standard Laboratory Atmosphere", 
               xlim=c(-3,3), 
               ylim=c(0,40),cex=0.5))#change xlim or ylim to frame it out. cex is dot size  

# Add colored points: red if padj<0.05, orange of log2FC>1, green if both)
with(subset(res, padj<.05 ), points(log2FoldChange, -log10(pvalue), pch=20, col="red",cex=0.5))
with(subset(res, abs(log2FoldChange)>1), points(log2FoldChange, -log10(pvalue), pch=20, col="orange",cex=0.5))
with(subset(res, padj<.05 & abs(log2FoldChange)>1), points(log2FoldChange, -log10(pvalue), pch=20, col="green",cex=0.5))

In [ ]:
%%R
res <- read.csv("./DEG/diesel_output.csv", header=TRUE, row.names=1)#Youll need to add in some data for this to work!

# Make a basic volcano plot
with(res, plot(log2FoldChange, -log10(padj), pch=20, 
               main="Diesel Exhaust vs Standard Laboratory Atmosphere", 
               xlim=c(-3,3), 
               ylim=c(0,40),cex=0.5))#change xlim or ylim to frame it out. cex is dot size  

# Add colored points: red if padj<0.05, orange of log2FC>1, green if both)
with(subset(res, padj<.05 ), points(log2FoldChange, -log10(padj), pch=20, col="red",cex=0.5))
with(subset(res, abs(log2FoldChange)>1), points(log2FoldChange, -log10(padj), pch=20, col="orange",cex=0.5))
with(subset(res, padj<.05 & abs(log2FoldChange)>1), points(log2FoldChange, -log10(padj), pch=20, col="green",cex=0.5))

#### Global heat maps
Global heat maps can be handy to look at the expression across multiple samples. At a quick glance, this will help you look at the raw data and cluster the samples out differently. These usually are based upon the hit count data, and will often include the controls for scaling. 

In this case we are using the heatmap.2 function as we did above, but instead of plotting out the distance matrix we will be plotting the hit counts. 

In [ ]:
%%R
data <- read.csv('./DEG/full_counts.csv', header=T, row.names=1)#make sure this data is just Gene in the first column followed by the columns of hitcounts
heatmap.2(as.matrix(top_n(data,500)), #selects just the columns of hit counts from the file, takes just the top 500 genes in this case. 
          scale=c('row'), #the direction to scale the information. For "row" this will more represent the differences across samples
          labRow='', 
          trace='none',
          col='bluered',#Changes the color to make it pretty
          Colv=TRUE, #Performs a standard hierarchal clustering for the columns (samples)
          Rowv=TRUE, #Likewise performs it for the rows (genes)
          cexRow=.75)

#### Focused heat maps
Usually you have a few genes of interest that you want to dial in on as you get further in to the analysis. This can be things like "I want to see how Actb is upregulated in my Drug X treated sample compared to control, and how its downregulated in my Drug Y treated sample compared to control". These can also be done similarly to the global heat maps above, or you can utilize other packages. 

#### Venn diagrams
Venn diagrams are a great way of comparing and contrasting samples as well. Often times you might think "what genes are common or unique to my samples". What better way to visualize this than with a venn diagram showing the overlap and what is unique. 

Here we are going to use a bit of python for fun. We will create a quick function to identify the significant genes for the samples. These lists of genes can then be fed directly into a matplot library called [matplotlib_venn](https://pypi.org/project/matplotlib-venn/) for plotting. Like the bash and R, there will be full python courses offered for anyone interested. 

In [ ]:
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
import csv, os

def counter(input_sheet):
    genes=[] #Empty list to populate later
    with open(input_sheet) as chart:
        reader=csv.DictReader(chart, delimiter=',') #Open the CSV file as a dictionary to parse the columns
        for row in reader: #Search through each row to then
           if str(row['padj']) != 'NA': #If it has a P value
               if float(row['padj']) <0.05: #If its significant. You can also use: if float(row['padj']) < 0.05 and float(row['log2fc']) > 1: as example
                   genes.append(row['']) #Add that significant gene to the list
    chart.close()
    return genes #Return the list of signficant genes.

urban_gene_list=counter('./DEG/urban_output.csv')
diesel_gene_list=counter('./DEG/diesel_output.csv')

venn2([set(urban_gene_list), set(diesel_gene_list)], set_labels = ('Urban Atmosphere','Diesel Exhaust'))
plt.title('Comparison of common or unique genes for each sample as compared to Standard Laboratory Air\n')

You can now see how there are some shared and some unique genes. From this we can start to infer that some genes are upregulated just in response to general pollutants, whereas others are more likely specific to pathways associated just with the unique features of the different pollutants. 

#### Gene ontology/Pathway analysis
Having individual genes is great, but the next step is nearly always related to groups of genes and how they could be inter-related. For the major organisms like Human/mouse/rat, there are a plethora of tools available. Things like []() and []() are commonly purchased for large scale licenses, with both available here at Rutgers. Regardless, there are also [gene ontologies]() for a much wider berth of organisms, as well as custom gene ontologies that can be manually curated and broad pan-bacterial ontologies. Fortunately with *E.coli*, there are available ontologies. 

Most of these tools rely on enrichment analysis. Quite simply, based on the number of genes inputed there are an expected number of genes. If your list has signficantly more or less than anticipated (based on z-score), then it is enriched. There are major limitations to this as it does not include direction in many cases and is only as good as your input list, so be careful of cherry picking data. If you also dont have enough genes, or enough genes in connected ontologies, you wont get any data out. 

This will follow the vignette [found here](https://github.com/tanghaibao/goatools/blob/master/notebooks/goea_nbt3102.ipynb), so if you want extra details go there! Since they have a full annotation for much of it Ill include just the code here for brevity. 

In [ ]:
from __future__ import print_function
from goatools.base import download_go_basic_obo
obo_fname = download_go_basic_obo()
from goatools.base import download_ncbi_associations
fin_gene2go = download_ncbi_associations()
from goatools.obo_parser import GODag
obodag = GODag("go-basic.obo")

from goatools.anno.genetogo_reader import Gene2GoReader

objanno = Gene2GoReader(fin_gene2go, taxids=[511145])#Taxa id for E.coli strain we are using
ns2assoc = objanno.get_ns2assc()
from gene_result import GENEID2NT as GeneID2nt_coli

from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS
goeaobj = GOEnrichmentStudyNS(
        GeneID2nt_coli.keys(), # List of protein-coding genes
        ns2assoc, # geneid/GO associations
        obodag, # Ontologies
        propagate_counts = False,
        alpha = 0.05, # default significance cut-off
        methods = ['fdr_bh']) # defult multipletest correction method

geneids_urban_study=urban_gene_list

goea_urban_results_all = goeaobj.run_study(geneids_urban_study, prt=None)
goea_urban_results_sig = [r for r in goea_urban_results_all if r.p_fdr_bh < 0.05]

goeaobj.wr_txt("urban_GO.txt", goea_urban_results_sig)

geneids_diesel_study=diesel_gene_list

goea_diesel_results_all = goeaobj.run_study(geneids_diesel_study, prt=None)
goea_diesel_results_sig = [r for r in goea_diesel_results_all if r.p_fdr_bh < 0.05]

goeaobj.wr_txt("diesel_GO.txt", goea_diesel_results_sig)

Now very unfortunately, this study does not have enough drastic alterations in gene expression to cause any major enrichment. So the output files here will be empty and it should show 0 pathways enriched. In that case you would have to more manually go through the data and identify trends. 

##### Basic bar charts
In most cases for gene ontologies, they are displayed as a basic bar chart. It is quite simply usually the y-axis as the ontology, and the x-axis as the number of genes enriched or -log(pvalue). 

##### Bubble plots
Bubble plots are the new craze for this, but are a bit more advanced. Ill just mention them here, but you can then plot the p-value, z-score, number of gene ratios, and directionality if you can include it. 

## Additional follow up for a [fun challenge](https://www.youtube.com/watch?v=dQw4w9WgXcQ)
If anyone dove into the paper this raw data is from, you'll see that there is another set of samples. These are *E.coli* that were first adapted to the diesel atmosphere (and acrued a few mutations) prior to the RNA-seq experiment. Using the different steps learned above, try downloading this new data set and processing it to compare how the diesel adapted samples vary from the wild type diesel samples. 